In [27]:
import pandas as pd
import os
from datetime import datetime
import datetime
import pyodbc
import sqlalchemy as sal


In [28]:
#pulling all .csv data to a dataframe

def fetching_data(folder_path):
    csv_files = []
    data_frames = []
    today_date = datetime.datetime.now().strftime('%Y%m%d')
    today_folder = os.path.join(folder_path, today_date)
    if not os.path.exists(today_folder):
        print(f'The Folder {today_folder} do not exist.')
        return None

    list_files = os.listdir(today_folder)

    for file in list_files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(today_folder,file))


    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        data_frames.append(df)

    
    if data_frames:
        df_combine_iphone = pd.concat(data_frames, ignore_index=True)
    
    return df_combine_iphone
    

df_combine_iphone = fetching_data(r'P:\Project\For Iphone\incoming')
    

In [29]:
df_combine_iphone

,Product Name,Product URL,Brand,Sale Price,Mrp,Number Of Ratings,Number Of Reviews,Upc,Star Rating,Ram
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",https://www.flipkart.com/apple-iphone-8-plus-g...,Apple,49900,49900,3431,356,MOBEXRGV7EHHTGUH,4.6,2 GB
1,"APPLE iPhone 8 Plus (Space Grey, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,Apple,84900,84900,3431,356,MOBEXRGVAC6TJT4F,4.6,2 GB
2,"APPLE iPhone 8 Plus (Silver, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,Apple,84900,84900,3431,356,MOBEXRGVGETABXWZ,4.6,2 GB
3,"APPLE iPhone 8 (Silver, 256 GB)",https://www.flipkart.com/apple-iphone-8-silver...,Apple,77000,77000,11202,794,MOBEXRGVMZWUHCBA,NaN,2 GB
4,"APPLE iPhone 8 (Gold, 256 GB)",https://www.flipkart.com/apple-iphone-8-gold-2...,Apple,77000,77000,11202,794,MOBEXRGVPK7PFEJZ,4.5,2 GB
...,...,...,...,...,...,...,...,...,...,...
57,"APPLE iPhone SE (Black, 64 GB)",https://www.flipkart.com/apple-iphone-se-black...,Apple,29999,39900,95909,8161,MOBFWQ6BR3MK7AUG,4.5,4 GB
58,"APPLE iPhone 11 (Purple, 64 GB)",https://www.flipkart.com/apple-iphone-11-purpl...,Apple,46999,54900,43470,3331,MOBFWQ6BTFFJKGKE,4.6,4 GB
59,"APPLE iPhone 11 (White, 64 GB)",https://www.flipkart.com/apple-iphone-11-white...,Apple,46999,54900,43470,3331,MOBFWQ6BVWVEH3XE,4.6,4 GB
60,"APPLE iPhone 11 (Black, 64 GB)",https://www.flipkart.com/apple-iphone-11-black...,Apple,46999,54900,43470,3331,MOBFWQ6BXGJCEYNY,4.6,4 GB


In [30]:
#making a copy
df_iphone = df_combine_iphone.copy(deep=True)

## Doing Some Changes in the Data (Transformation)

In [31]:
df_iphone[['Brand','Model Details']] = df_iphone['Product Name'].apply(lambda x: pd.Series(x.split('(',1)))

In [32]:
df_iphone[['Color','Storage']] = df_iphone['Model Details'].apply(lambda x: pd.Series(x.split(',',1)))

In [33]:
columns_list = ['Model Details','Storage']
for col in columns_list:
    df_iphone[col] = df_iphone[col].apply(lambda x: pd.Series(x.rstrip(')')))

In [34]:
#adding new column "Standard Model"

def iphone_main_model_name(brand):
    if 'iPhone 8' in brand:
        return 'iphone 8'
    elif 'iPhone XS' in brand:
        return 'iPhone XS'
    elif 'iPhone XR' in brand:
        return 'iPhone XR'
    elif 'iPhone 11' in brand:
        return 'iPhone 11'
    elif 'iPhone SE' in brand:
        return 'iPhone SE'
    else:
        return 'iPhone 12'

#df_iphone['Standard_Model'] = iphone_main_model_name(df_iphone['Brand'])
df_iphone["Standard_Model"] = df_iphone["Brand"].apply(iphone_main_model_name)

In [35]:
df_iphone.drop(['Model Details'], axis=1, inplace=True)

In [36]:
df_iphone['Color'] = df_iphone['Color'].apply(lambda x: x.capitalize())

In [37]:
df_iphone.rename(columns = {'Product Name':'Product_Name', 'Product URL':'Product_URL', 'Sale Price':'Sale_Price', 'Number Of Ratings':'Number_Of_Ratings', 'Number Of Reviews':'Number_Of_Reviews', 'Star Rating':'Star_Rating', 'Standard Model':'Standard_Model'}, inplace=True)

In [38]:
#avg_rating = df_iphone['Star_Rating'].mean()
#df_iphone['Star_Rating'].fillna(value = avg_rating, inplace=True)

In [39]:
ram_wise_avg_rating = df_iphone.groupby(['Ram'])['Star_Rating'].mean()
ram_wise_avg_rating

Ram
2 GB    4.541667
3 GB    4.600000
4 GB    4.589286
6 GB    4.577778
Name: Star_Rating, dtype: float64

In [40]:
df_iphone

,Product_Name,Product_URL,Brand,Sale_Price,Mrp,Number_Of_Ratings,Number_Of_Reviews,Upc,Star_Rating,Ram,Color,Storage,Standard_Model
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",https://www.flipkart.com/apple-iphone-8-plus-g...,APPLE iPhone 8 Plus,49900,49900,3431,356,MOBEXRGV7EHHTGUH,4.6,2 GB,Gold,64 GB,iphone 8
1,"APPLE iPhone 8 Plus (Space Grey, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,APPLE iPhone 8 Plus,84900,84900,3431,356,MOBEXRGVAC6TJT4F,4.6,2 GB,Space grey,256 GB,iphone 8
2,"APPLE iPhone 8 Plus (Silver, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,APPLE iPhone 8 Plus,84900,84900,3431,356,MOBEXRGVGETABXWZ,4.6,2 GB,Silver,256 GB,iphone 8
3,"APPLE iPhone 8 (Silver, 256 GB)",https://www.flipkart.com/apple-iphone-8-silver...,APPLE iPhone 8,77000,77000,11202,794,MOBEXRGVMZWUHCBA,NaN,2 GB,Silver,256 GB,iphone 8
4,"APPLE iPhone 8 (Gold, 256 GB)",https://www.flipkart.com/apple-iphone-8-gold-2...,APPLE iPhone 8,77000,77000,11202,794,MOBEXRGVPK7PFEJZ,4.5,2 GB,Gold,256 GB,iphone 8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,"APPLE iPhone SE (Black, 64 GB)",https://www.flipkart.com/apple-iphone-se-black...,APPLE iPhone SE,29999,39900,95909,8161,MOBFWQ6BR3MK7AUG,4.5,4 GB,Black,64 GB,iPhone SE
58,"APPLE iPhone 11 (Purple, 64 GB)",https://www.flipkart.com/apple-iphone-11-purpl...,APPLE iPhone 11,46999,54900,43470,3331,MOBFWQ6BTFFJKGKE,4.6,4 GB,Purple,64 GB,iPhone 11
59,"APPLE iPhone 11 (White, 64 GB)",https://www.flipkart.com/apple-iphone-11-white...,APPLE iPhone 11,46999,54900,43470,3331,MOBFWQ6BVWVEH3XE,4.6,4 GB,White,64 GB,iPhone 11
60,"APPLE iPhone 11 (Black, 64 GB)",https://www.flipkart.com/apple-iphone-11-black...,APPLE iPhone 11,46999,54900,43470,3331,MOBFWQ6BXGJCEYNY,4.6,4 GB,Black,64 GB,iPhone 11


In [41]:
for index, row in df_iphone.iterrows():
    if pd.isnull(row['Star_Rating']):
        ram = row['Ram']
        avg_rating = ram_wise_avg_rating[ram]
        df_iphone.at[index, 'Star_Rating'] = avg_rating

In [42]:
df_iphone['Discount_Percentage'] = ((df_iphone['Mrp'] - df_iphone['Sale_Price'])/ df_iphone['Mrp']) * 100

## Loading Clean Data To SQL

In [43]:
engine = sal.create_engine('mssql://DESKTOP-EAQGBSR\SQLEXPRESS/python?driver=ODBC+Driver+17+for+SQL+Server')
connection_est = engine.connect()

df_iphone.to_sql('df_iphone', con=connection_est, index=False, if_exists='replace')

62

## Extracting SQL Table for Analysis

In [44]:
df_sql_iphone = pd.read_sql_query('select * from [dbo].[df_iphone]',connection_est)

### Q: Which model has highest percent discount ?

In [45]:
df_max_brand_disc =  df_sql_iphone.groupby(['Brand'], as_index=False)['Discount_Percentage'].max().sort_values('Discount_Percentage', ascending=False).iloc[0]
df_max_brand_disc

Brand                  APPLE iPhone 11 Pro 
Discount_Percentage               29.644465
Name: 1, dtype: object

### Q: find total no of models  each space configuration (128 GB , 64 GB etc)

In [46]:
df_storage_wise = df_sql_iphone.groupby('Storage').size().reset_index(name='total')
df_storage_wise

,Storage,total
0,128 GB,24
1,256 GB,14
2,512 GB,4
3,64 GB,20


### Q: find total number of models for each color

In [48]:
df_color_wise = df_sql_iphone.groupby(['Color']).size().reset_index(name='Total')
df_color_wise

,Color,Total
0,Black,10
1,Blue,2
2,Coral,1
3,Gold,6
4,Graphite,4
5,Green,1
6,Midnight green,4
7,Pacific blue,4
8,Purple,1
9,Red,6


### Q: find total number of models by iphone versions

In [54]:
df_iphone_main_models = df_sql_iphone.groupby(['Standard_Model']).size().reset_index(name='count')
df_iphone_main_models

,Standard_Model,count
0,iPhone 11,16
1,iPhone 12,26
2,iPhone SE,6
3,iPhone XR,5
4,iPhone XS,1
5,iphone 8,8


### Q: list top 5 models having highest number of reviews

In [64]:
top_iphone_reviews = df_sql_iphone.groupby(['Brand']).size().reset_index(name='count').sort_values('count', ascending=False).iloc[0:5]
top_iphone_reviews

,Brand,count
6,APPLE iPhone 12 Pro Max,8
0,APPLE iPhone 11,7
3,APPLE iPhone 12,7
4,APPLE iPhone 12 Mini,6
2,APPLE iPhone 11 Pro Max,5


### Q: what is the price difference between highest price and lowest price iphone (based on mrp)

In [68]:
high_price_iphone = df_sql_iphone['Mrp'].max()
low_price_iphone = df_sql_iphone['Mrp'].min()
price_difference = high_price_iphone - low_price_iphone
price_difference

110000

### Q: find total no of reviews for iphone 11 and iphone 12 category . Output should have only 2 rows (for 11 and 12)

In [74]:
filter_df = df_sql_iphone[df_sql_iphone['Standard_Model'].isin(['iPhone 11', 'iPhone 12'])]
df_review_count = filter_df.groupby(['Standard_Model'])['Number_Of_Reviews'].sum().reset_index(name='total')
df_review_count

,Standard_Model,total
0,iPhone 11,25965
1,iPhone 12,2208


### Q: which iphone has 3rd highest MRP

In [79]:
iphone_mrp = df_sql_iphone.groupby(['Brand'])['Mrp'].max().reset_index(name='MRP').sort_values('MRP',ascending=False).iloc[2]
iphone_mrp

Brand    APPLE iPhone 12 Pro Max 
MRP                        139900
Name: 6, dtype: object

### Q: what is the average mrp of iphones which costs above 100,000

In [91]:
df_above_onelakh = df_sql_iphone[df_sql_iphone['Mrp']>100000]
df_above_onelakh.groupby(['Standard_Model'])['Mrp'].mean()

Standard_Model
iPhone 11    124844.444444
iPhone 12    134515.384615
Name: Mrp, dtype: float64